In [1]:
import os
import cv2
import pandas
import numpy as np
import matplotlib.pyplot as plt

def extract_image(f):
    im = cv2.imread(f)
    im = cv2.cvtColor(im, cv2.COLOR_BGR2YUV)
    im = im[:,:,0]
    mn, mx = np.min(im), np.max(im)
    a, b = 0., 1.
    im = a + ((im-mn)/(mx-mn))*(b-a)
    return im

def process_db(dir):
    with open(os.path.join(dir, 'driving_log.csv'), 'r') as f:
        data = pandas.read_csv(f, header=0)
        res = np.asarray(data.values)
        c, l, r, st, t, b, sp = np.split(res, 7, axis=1)
        c = np.array([extract_image(os.path.join(dir, p)) for p in c.squeeze()])
        return c, st.squeeze()
        
c, st = process_db('./data')
print(c.shape)
%matplotlib inline
#plt.imshow(c[20], cmap='gray')
        

(8036, 160, 320)


In [32]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

c, st = shuffle(c, st)
st = st.astype(np.float32)
c = np.reshape(c, [-1, 160, 320, 1])
print(c.shape)


X_train, y_train = c, st
print(X_train.shape, y_train.shape)



(8036, 160, 320, 1)
(8036, 160, 320, 1) (8036,)


In [33]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers.convolutional import Convolution2D
from keras.wrappers.scikit_learn import KerasRegressor

from keras.layers.pooling import MaxPooling2D
import numpy as np
from random import random

def my_model():
    model = Sequential()
    model.add(Convolution2D(16, 5, 5, border_mode='same', activation='relu', name='c1', input_shape=(160, 320, 1)))
    model.add(MaxPooling2D((2,2)))
    model.add(Convolution2D(32, 5, 5, border_mode='same', activation='relu', name='c2'))
    model.add(MaxPooling2D((2,2)))
    model.add(Convolution2D(64, 5, 5, border_mode='same', activation='relu', name='c3'))
    model.add(MaxPooling2D((2,2)))

    model.add(Flatten())
    model.add(Dense(64, name='d1',activation='relu'))
    model.add(Dense(32, name='d2',activation='linear'))
    model.add(Dense(16, name='d3',activation='relu'))
    model.add(Dense(1, name='d4', activation='linear'))
    
    return model


model = my_model()
model.compile('adam', 'mse', ['mse'])
history = model.fit(X_train, y_train, batch_size=32, nb_epoch=30, verbose=2, validation_split=0.2)
y_pred = model.predict(X_train)



Train on 6428 samples, validate on 1608 samples
Epoch 1/30
45s - loss: 0.0164 - mean_squared_error: 0.0164 - val_loss: 0.0105 - val_mean_squared_error: 0.0105
Epoch 2/30
43s - loss: 0.0100 - mean_squared_error: 0.0100 - val_loss: 0.0083 - val_mean_squared_error: 0.0083
Epoch 3/30
43s - loss: 0.0090 - mean_squared_error: 0.0090 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 4/30
43s - loss: 0.0085 - mean_squared_error: 0.0085 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 5/30
43s - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0085 - val_mean_squared_error: 0.0085
Epoch 6/30
43s - loss: 0.0072 - mean_squared_error: 0.0072 - val_loss: 0.0081 - val_mean_squared_error: 0.0081
Epoch 7/30
43s - loss: 0.0066 - mean_squared_error: 0.0066 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 8/30
43s - loss: 0.0059 - mean_squared_error: 0.0059 - val_loss: 0.0084 - val_mean_squared_error: 0.0084
Epoch 9/30
43s - loss: 0.0052 - mean_squared_error: 0.0052 - val

In [34]:
print(np.column_stack((y_train, y_pred.squeeze())))
print(model.save("model.8000.h5"))

[[ 0.         -0.00798936]
 [ 0.17658231  0.16959016]
 [-0.04076847 -0.02267045]
 ..., 
 [ 0.         -0.01274112]
 [-0.04076847  0.00534799]
 [-0.09773462 -0.10673321]]
None
